N-gram'ler duygu analizi (sentiment analysis) için kullanılabilir:

In [ ]:
import pandas as pd
import nltk
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score

# Gerekli nltk verilerini indirme
nltk.download('punkt')

# Eğitim verilerini CSV dosyasından okuma
def csv_den_veri_yukle(dosya_adi):
    # CSV dosyasını pandas ile yükle
    df = pd.read_csv(dosya_adi)
    
    # "text" ve "label" sütunlarını kullan
    metinler = df['text'].values
    etiketler = df['label'].values
    
    return metinler, etiketler

# Büyük metin dosyasını okuma ve N-gram özelliklerini çıkarma
def ngram_vektorizer_olustur(dosya_adi, ngram_range=(1, 2)):
    with open(dosya_adi, "r", encoding="utf-8") as file:
        metin = file.read()
    
    # Metni cümle seviyesinde tokenize et
    cumleler = nltk.sent_tokenize(metin)
    
    # CountVectorizer ile N-gram'leri çıkar ve dönüştürücü olarak geri döndür
    vectorizer = CountVectorizer(ngram_range=ngram_range, stop_words='english')
    vectorizer.fit(cumleler)
    
    return vectorizer

# Eğitim verilerini CSV dosyasından yükleme
metinler, etiketler = csv_den_veri_yukle("sentiment_train.csv")

# N-gram özellikleri çıkaran vektörleştiriciyi büyük metin dosyasından öğrenme
vectorizer = ngram_vektorizer_olustur("odtu_derlemi.txt", ngram_range=(1, 2))

# Eğitim verilerinde özellik çıkarımı
X = vectorizer.transform(metinler)

# Eğitim ve test veri setlerine ayırma
X_train, X_test, y_train, y_test = train_test_split(X, etiketler, test_size=0.3, random_state=42)

# Naive Bayes modelini kullanarak eğitim yapma
model = MultinomialNB()
model.fit(X_train, y_train)

# Test verileri ile tahmin yapma
y_pred = model.predict(X_test)

# Sonuçların doğruluğunu kontrol etme
accuracy = accuracy_score(y_test, y_pred)
print(f"Doğruluk: {accuracy:.2f}")

# Yeni cümleler için duygu tahmini yapma
yeni_cumleler = [
    "Bu ürün çok güzel",
    "Film berbattı, zaman kaybıydı",
    "Yemekler çok lezzetliydi",
    "Hizmet berbat ve yavaş",
    "Bilgisayarların içinde işlemci vardır",
    "Neresinden tutsan elinde kalıyor",
    "Hangi fişi çekmem gerekiyordu",
    "Keşke hafızamı kaybetsem de bu kitabı baştan okuyabilsem"
]

# Yeni cümleleri N-gram vektörüne dönüştürme ve tahmin etme
yeni_ozellikler = vectorizer.transform(yeni_cumleler)
yeni_tahminler = model.predict(yeni_ozellikler)

# Yeni cümlelerin tahminlerini gösterme
for cumle, tahmin in zip(yeni_cumleler, yeni_tahminler):
    print(f"Cümle: '{cumle}' - Tahmin Edilen Duygu: {tahmin}")
